# 🧪 WFGY Diagnostic Notebook (v0.5 — widget-free)
Measure **ΔS** and classify common reasoning failures with one click.
No API key • No widgets • Works in any Colab session.

## 📐 Core formula
$$\Delta S = 1 - \cos\theta\bigl(I, G\bigr)$$  
*$I$ = current-step embedding, $G$ = ground-truth / prompt embedding.*

In [ ]:
!pip -q install sentence-transformers --upgrade

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def delta_s(a_vec, b_vec):
    return 1 - util.cos_sim(a_vec, b_vec).item()

def classify_failure(dS):
    if dS > 0.80:
        return ('No.1 Hallucination / Chunk Drift', 'Apply BBMC + boundary check')
    if dS > 0.60:
        return ('No.2 Interpretation Collapse', 'Insert λ_observe checkpoint')
    if dS > 0.40:
        return ('No.6 Logic Collapse', 'Trigger BBCR fallback')
    if dS > 0.25:
        return ('No.9 Entropy Collapse', 'Use BBAM attention modulation')
    return ('Stable (ΔS ≤ 0.25)', 'No action — pass')

## ✏️ Edit & run
Replace the text in **Prompt** / **Answer**, press ▶️ to diagnose.

In [ ]:
prompt = "### Product Review\nThe new phone has a crystal-clear display and fast charging…"
answer = "The capital of France is Paris. Also, your phone seems great!"

# --- diagnostic ---
dS = delta_s(model.encode(prompt, convert_to_tensor=True),
             model.encode(answer, convert_to_tensor=True))

failure, fix = classify_failure(dS)
print("ΔS :", f"{dS:.3f}")
print("Failure detected :", failure)
print("Suggested fix    :", fix)

---
### Next Steps  
* Explore **Problem Map 1.0** & **2.0** for full 16 failure modes.  
* Read about **λ_observe, E_resonance, BBCR** in the WFGY 1.0 paper.  
* Fork & extend this notebook — MIT-licensed.  

> Repo → https://github.com/onestardao/WFGY